In [4]:
# %pip install jupyter_dash
# %pip install dash
# %pip install pandas_datareader
from ipywidgets import interact
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from pandas_datareader import data as web





def f(Dataset):
    df1 = pd.read_csv('Annual_Count_of_Visitors_purpose_in_Taiwan.csv',thousands=',',encoding='Big5') 
    df2 = pd.read_csv('Annual_Age_Count_of_Visitors_in_Taiwan.csv',thousands=',',encoding='Big5')
    df3 = pd.read_csv('2015_Count_of_Visitors_in_Taiwan(by_sex).csv',encoding='utf-8') 
    df4 = pd.read_csv('Annual_Count_of_Visitors_in_Taiwan.csv',encoding='Big5')

    control = Dataset
    if control == '歷年來台旅客目的統計':
#         df1 = pd.read_csv('Annual_Count_of_Visitors_purpose_in_Taiwan.csv',thousands=',',encoding='Big5') 
        fig = go.Figure()
        df1=df1[df1["細分"].isin(["總計 Grand Total"])]
         # df=df[df["年別"]==2018]
        Year = df1["年別"]
        business=df1["業務"]
        Tourist=df1["觀光"]
        Meeting=df1["會議"]
        Study=df1["求學"]
        Medical=df1["醫療"]
        Else=df1["其他"]
        fig.add_trace(go.Scatter(x=Year, y=business,
                    mode='lines+markers',
                    name='業務'))
        fig.add_trace(go.Scatter(x=Year, y=Tourist,
                    mode='lines+markers',
                    name='觀光'))
        fig.add_trace(go.Scatter(x=Year, y=Meeting,
                    mode='lines+markers',
                    name='會議'))
        fig.add_trace(go.Scatter(x=Year, y=Study,
                    line = dict(color='royalblue', width=4, dash='dash'),
                    name='求學'))
        fig.add_trace(go.Scatter(x=Year, y=Medical,
                    mode='lines',
                    name='醫療'))
        fig.add_trace(go.Scatter(x=Year, y=Else,
                    line = dict(color='firebrick', width=4, dash='dot'),
                    name='其他'))
        fig.update_layout(title="歷年來台旅客目的統計")
        fig.show()
    elif control== '來台旅客人數統計(以國家區分)':
        df1 = pd.read_csv('Annual_Count_of_Visitors_purpose_in_Taiwan.csv',thousands=',',encoding='Big5') 
        fig = px.choropleth(df1,
                    locationmode="country names",locations="細分",
                    color='合計', # column by which to color-code
                    hover_name="細分",animation_frame="年別"
                    ,range_color=(1000,500000),color_continuous_scale=px.colors.sequential.Turbo)
        fig.update_layout(
        # add a title text for the plot
        title_text = '來台旅客人數統計(以國家區分)'
        )
        fig.show()
    elif control == '歷年來台旅客人數總計成長率':
#         df2 = pd.read_csv('Annual_Count_of_Visitors_in_Taiwan.csv',encoding='Big5')
        fig = px.line(df4, x='Year', y='TotalGross')
        fig.update_layout(title="歷年來台旅客人數總計成長率")
        fig.show()
    elif control == '來台性別比例':
        app = JupyterDash('Hello World')
#         df = pd.read_csv('2015_Count_of_Visitors_in_Taiwan(by_sex).csv',encoding='utf-8') 
        all_year =df3["年別"].unique()
        app.layout = html.Div([
        dcc.Dropdown(
        id="Year_dropdown",
        options=[{"label": x, "value": x} 
                 for x in all_year],
        value=all_year[1],
        clearable=False,
        ),
           dcc.Graph(id='the_graph')
        ], style={'width': '500'})
        @app.callback(Output('the_graph', 'figure'), [Input('Year_dropdown', 'value')])
        def update_graph(years):
            mask = df3["年別"]==years
            data=df3[mask]
            fig = px.bar(data,x="細分",y="男占合計百分比",title="來台性別比")
            fig.add_trace(go.Bar(x=data["細分"], y=data["女占合計百分比"],name='女占合計百分比'))
            fig.update_layout(barmode='group',xaxis_tickangle=-60,showlegend=False)
            return fig

        app.run_server(mode='inline')
    elif control == '歷年來台地區統計':
#         df3 = pd.read_csv('2015_Count_of_Visitors_in_Taiwan(by_sex).csv',thousands=',',encoding='utf-8')
        all_continents =df3["居住地"].unique()

        app = JupyterDash(__name__)


        app.layout = html.Div([
        dcc.Checklist(
        id="checklist",
        options=[{"label": x, "value": x} 
                 for x in all_continents],
        value=all_continents[5:],
        labelStyle={'display': 'inline-block'}
        ),
        dcc.Graph(id="line-chart"),
        ])
        @app.callback(
        Output("line-chart", "figure"), 
        [Input("checklist", "value")])
        def update_line_chart(continents):
            mask = df3["居住地"].isin(continents)
            fig = px.line(df3[mask],
                x="年別", y="合計", color='細分')
            return fig
        app.run_server(mode='inline')
    elif control =='歷年來台旅客統計':
        app = JupyterDash('Hello World')

        app.layout = html.Div([
            dcc.Dropdown(
                id='my-dropdown',
                options=[
                    {'label': 'TotalCount', 'value': 'TotalCount'},
                    {'label': 'ForeignTravelers', 'value': 'ForeignTravelers'},
                    {'label': 'OverseasChineseTourists', 'value': 'OverseasChineseTourists'}
                ],
                value='TotalCount'
            ),
            dcc.Graph(id='my-graph')
        ], style={'width': '500'})

        @app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
        def update_graph(selected_dropdown_value):
#             df2 = pd.read_csv('Annual_Count_of_Visitors_in_Taiwan.csv',encoding='Big5')
            return {
                'data': [{
                    'x': df4.Year,
                    'y': df4[selected_dropdown_value]
                }],
                'layout': {'margin': {'l': 40, 'r': 0, 't': 20, 'b': 30}}
            }
        
        app.run_server(mode='inline')
    elif control =='歷年來台旅客年齡統計':
        app = JupyterDash('Hello World')
#         df = pd.read_csv('Annual_Age_Count_of_Visitors_in_Taiwan.csv',thousands=',',encoding='Big5')
        all_continents =df2["細分"].unique()
        app.layout = html.Div([
              html.Div([
                    dcc.Dropdown(
                        id='my-dropdown',
                        options=[
                            {'label': '9歲及以下', 'value': '9歲及以下'},
                            {'label': '10至19歲', 'value': '10至19歲'},
                            {'label': '20至29歲', 'value': '20至29歲'},
                            {'label': '30至39歲', 'value': '30至39歲'},
                            {'label': '40至49歲', 'value': '40至49歲'},
                            {'label': '50至59歲', 'value': '50至59歲'},
                            {'label': '60歲以上', 'value': '60歲以上'},
                            {'label': '合計','value':'合計'}
                        ],
                        value='合計'
                    ),
              ]),
                dcc.Dropdown(
                id="AREA_dropdown",
                options=[{"label": x, "value": x} 
                         for x in all_continents],
                value=all_continents[1],
                clearable=False,
                ),
                   dcc.Graph(id='the_graph')
                ], style={'width': '500'})
        @app.callback(Output('the_graph', 'figure'), [Input('my-dropdown', 'value'),Input('AREA_dropdown', 'value')])
        def update_graph(selected_dropdown_value,continents):
            mask = df2["細分"]==continents
            fig = px.bar(df2[mask],x="年別", y=selected_dropdown_value,color="年別")
            return fig

        app.run_server(mode='inline')
    

    return Dataset
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
interact(f, Dataset = widgets.RadioButtons(
options=['歷年來台旅客目的統計', '來台旅客人數統計(以國家區分)','歷年來台旅客人數總計成長率','來台性別比例','歷年來台地區統計','歷年來台旅客統計','歷年來台旅客年齡統計'],
description='統計選擇:',
disabled=False))

Note: you may need to restart the kernel to use updated packages.


interactive(children=(RadioButtons(description='統計選擇:', options=('歷年來台旅客目的統計', '來台旅客人數統計(以國家區分)', '歷年來台旅客人數總計成…

<function __main__.f(Dataset)>